<div style="color: #40E0D0; font-family: Cambria, serif; text-align: center; margin: 20px;">

<div style="font-size: 32px; font-weight: bold; margin-bottom: 15px;">Callable Contingent Income Securities due December 3, 2025</div>

<div style="font-size: 18px; margin-bottom: 15px;">
<a href="https://sec.gov/Archives/edgar/data/1666268/000183988223032004/ms75_424b2-16958.htm" style="color: #40E0D0;">SEC Filing Link to the Security
</a>
</div>

<div style="font-size: 24px; font-weight: bold; margin-bottom: 15px;">Payments on the Securities Based on the Worst Performing of the Russell 2000® Index and the S&P 500® Index</div>

<div style="font-style: italic; font-size: 16px;">-Valuation report Python notebook by Yadvesh, Krish and Mayank</div>

</div>

#### Group Members (NetID)
* Krish Desai (kcdesai2)
* Mayank Kalra (mkalra2)
* Yadvesh Yadav (yyada)

---

<div style="border-bottom: 3px solid black; display: inline-block;">
<span style="font-family: Cambria, serif; font-weight: bold; font-size: 28px;">I) Introduction ==></span>
</div>

<div style="border-bottom: 2px solid black; display: inline-block;">
<span style="font-family: Cambria, serif; font-weight: bold; font-size: 20px;">1) Below are the Key Dates of the Note:</span>
</div>

![](./images_project_2/key_dates.png)

<div style="border-bottom: 2px solid black; display: inline-block;">
<span style="font-family: Cambria, serif; font-weight: bold; font-size: 20px;">2) Below are the Note Offering:</span>
</div>

![](./images_project_2/note_offering.png)

<div style="border-bottom: 2px solid black; display: inline-block;">
<span style="font-family: Cambria, serif; font-weight: bold; font-size: 20px;">3) Below are the Observation Dates and Coupon Payment Dates:</span>
</div>

![](./images_project_2/obs_cpn_dates.png)

---

<div style="border-bottom: 3px solid black; display: inline-block;">
<span style="font-family: Cambria, serif; font-weight: bold; font-size: 28px;">II) Bloomberg Data (Risk-Free Rates(OIS), Dividend Yields, Volatality Matrices and Correlation Factors) is as follows ==> </span>
</div>

<div style="border-bottom: 2px solid black; display: inline-block;">
<span style="font-family: Cambria, serif; font-weight: bold; font-size: 20px;">1) Risk Free Rate (OIS Rate):</span>
</div>

![](./images_project_2/ois_rates.png)

---

<div style="border-bottom: 2px solid black; display: inline-block;">
<span style="font-family: Cambria, serif; font-weight: bold; font-size: 20px;">2) Dividend Yields for both indices:</span>
</div>

<table style="width:80%; margin:auto; border-collapse: separate; border-spacing: 80px 0;"><tr>
<td style="width:50%; border:none;">
    <div style="border-bottom: 2px solid black; display: inline-block;">
        <span style="font-family: Cambria, serif; font-weight: bold; font-size: 18px;">2.1) Russell 2000® Index (RTY):</span>
    </div>
    <img src="./images_project_2/rty_div.png" style="width:100%">
</td>
<td style="width:50%; border:none;">
    <div style="border-bottom: 2px solid black; display: inline-block;">
        <span style="font-family: Cambria, serif; font-weight: bold; font-size: 18px;">2.2) S&P 500® Index (SPX):</span>
    </div>
    <img src="./images_project_2/spx_div.png" style="width:100%">
</td>
</tr></table>

---

<div style="border-bottom: 2px solid black; display: inline-block;">
<span style="font-family: Cambria, serif; font-weight: bold; font-size: 20px;">3) Implied Volatality Matrices for both indices:</span>
</div>

<table style="width:100%; margin:auto;"><tr>
<td style="width:100%; border:none; padding-bottom: 30px;">
    <div style="border-bottom: 2px solid black; display: inline-block;">
        <span style="font-family: Cambria, serif; font-weight: bold; font-size: 18px;">3.1) Russell 2000® Index (RTY):</span>
    </div>
    <img src="./images_project_2/rty_vol.png" style="width:100%">
</td>
</tr>
<tr>
<td style="width:100%; border:none;">
    <div style="border-bottom: 2px solid black; display: inline-block;">
        <span style="font-family: Cambria, serif; font-weight: bold; font-size: 18px;">3.2) S&P 500® Index (SPX):</span>
    </div>
    <img src="./images_project_2/spx_vol.png" style="width:100%">
</td>
</tr></table>

---

<div style="border-bottom: 2px solid black; display: inline-block;">
<span style="font-family: Cambria, serif; font-weight: bold; font-size: 28px;">III) Getting the Risk-free rates from discount factors above ==></span>
</div>

In [1]:
# Importing Necessary Libraries
from datetime import datetime
import calendar
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [2]:
# function to get fractional difference between two dates
def get_year_fraction(start_date, end_date):
    if isinstance(start_date, str):
        start_date = datetime.strptime(start_date, "%Y-%m-%d")
    if isinstance(end_date, str):
        end_date = datetime.strptime(end_date, "%Y-%m-%d")
    
    total_years = 0.0

    current_date = start_date
    while current_date < end_date:
        year = current_date.year
    
        if year == end_date.year:
            period_end = end_date
        else:
            period_end = datetime(year + 1, 1, 1)

        days_in_period = (period_end - current_date).days
        days_in_year = 366 if calendar.isleap(year) else 365
        total_years += days_in_period / days_in_year
        current_date = period_end
    
    return total_years

In [3]:
# Key dates
T_0 = "2023-11-28"
T_1 = "2023-12-01"
T_2 = "2025-11-28"
T_3 = "2025-12-03"

# Discount Factors at Key Dates
DF_0 = 1
DF_2 = 0.911000

# DF_1 and DF_3 needs to be calculated using Interpolation
# We have:
dt_2023_12_05 = "2023-12-05"
DF_2023_12_05 = 0.998964
dt_2026_11_30 = "2026-11-30"
DF_2026_11_30 = 0.877524

# Now using Interpolation we have:
DF_1 = DF_0 + (DF_2023_12_05 - DF_0)*(get_year_fraction(T_0, T_1))/(get_year_fraction(T_0, dt_2023_12_05))
print(f"The Value of DF_1 is: {DF_1:.6f}\n")
DF_3 = DF_2 + (DF_2026_11_30 - DF_2)*(get_year_fraction(T_2, T_3))/(get_year_fraction(T_2, dt_2026_11_30))
print(f"The Value of DF_3 is: {DF_3:.6f}")

The Value of DF_1 is: 0.999556

The Value of DF_3 is: 0.910544


<div style="border-bottom: 2px solid black; display: inline-block; margin-bottom: 20px;">
<span style="font-family: Cambria, serif; font-weight: bold; font-size: 20px;">The Discount Factor Table after Linear Interpolation Looks like this</span>
</div>

<div style="font-family: Cambria, serif;">

| Keys | Dates     | DF        |
|------|-----------|-----------|
| T_0  | 11/28/2023  | 1.000000  |
| T_1  | 12/01/2023 | 0.999556  |
| T_2  | 11/28/2025 | 0.911000  |
| T_3  | 12/03/2025 | 0.910544  |

</div>

In [4]:
# function to get r(0,T) from the discount factors DF
def r_from_DF(DF, T, T_0):
    dt = get_year_fraction(T_0, T)
    r = -np.log(DF)/dt
    return r, dt

In [5]:
r_T0_T1, T0_T1 = r_from_DF(DF_1, T_1, T_0)
print(f"The Value of continuously compounded risk free rate between T_0 and T_1(r_T0_T1) is: {r_T0_T1:.8f}\n")
r_T0_T2, T0_T2 = r_from_DF(DF_2, T_2, T_0)
print(f"The Value of continuously compounded risk free rate between T_0 and T_2(r_T0_T2) is: {r_T0_T2:.8f}\n")
r_T0_T3, T0_T3 = r_from_DF(DF_3, T_3, T_0)
print(f"The Value of continuously compounded risk free rate between T_0 and T_3(r_T0_T3) is: {r_T0_T3:.8f}\n")

The Value of continuously compounded risk free rate between T_0 and T_1(r_T0_T1) is: 0.05403200

The Value of continuously compounded risk free rate between T_0 and T_2(r_T0_T2) is: 0.04660619

The Value of continuously compounded risk free rate between T_0 and T_3(r_T0_T3) is: 0.04653782



<div style="border-bottom: 2px solid black; display: inline-block; margin-bottom: 20px;">
<span style="font-family: Cambria, serif; font-weight: bold; font-size: 20px;">The Discount Factor Table after Linear Interpolation Looks like this</span>
</div>

<div style="font-family: Cambria, serif;">

| Keys | Dates     | DF        | delta(T_x, T_0) | r(0,T_x)   |
|------|-----------|-----------|-----------------|------------|
| T_0  | 11/28/2023  | 1.000000  | 0.00000000     | 0.00000000 |
| T_1  | 12/01/2023 | 0.999556  | 0.00821918     | 0.05403200 |
| T_2  | 11/28/2025 | 0.911000  | 2.00000000     | <span style="background-color: green;">0.04660619</span> |
| T_3  | 12/03/2025 | 0.910544  | 2.01369863     | 0.04653782 |

</div>

In [6]:
# now we need to calculate teh forward rate F(T_1,T_3)
T1_T3 = get_year_fraction(T_1, T_3)
F_T1_T3 = ((r_T0_T3*T0_T3) - (r_T0_T1*T0_T1))/T1_T3
print(f"The Value of continuously compounded Forward rate between T_1 and T_3(F_T1_T3) is: {F_T1_T3:.8f}\n")

The Value of continuously compounded Forward rate between T_1 and T_3(F_T1_T3) is: 0.04650710



---

<div style="border-bottom: 2px solid black; display: inline-block;">
<span style="font-family: Cambria, serif; font-weight: bold; font-size: 28px;">IV) Calculating the correlation between indices (RTY and SPX) for 6M, 12M and 24M time horizons==></span>
</div>

In [7]:
# function to read historical csv and get returns based on index values
def historical_df(file_path):
    df = pd.read_csv(file_path)
    df.set_index('Date', inplace=True)

    if file_path=='./data_project_2/rty_hp.csv':
        df.rename(columns={'px_last': 'px_last_rty'}, inplace=True)
        df['px_last_rty'] = pd.to_numeric(df['px_last_rty'])
        df['px_last_rty_previous'] = df['px_last_rty'].shift(-1)
        df['diff'] = df['px_last_rty'] - df['px_last_rty_previous']
        df['pct_change_rty'] = df['diff']/np.abs(df['px_last_rty_previous'])*100
    else:
        df.rename(columns={'px_last': 'px_last_spx'}, inplace=True)
        df['px_last_spx'] = pd.to_numeric(df['px_last_spx'])
        df['px_last_spx_previous'] = df['px_last_spx'].shift(-1)
        df['diff'] = df['px_last_spx'] - df['px_last_spx_previous']
        df['pct_change_spx'] = df['diff']/np.abs(df['px_last_spx_previous'])*100
    
    df = df.iloc[:,[0,3]]
   
    return df

In [8]:
df_rty = historical_df('./data_project_2/rty_hp.csv')
df_spx = historical_df('./data_project_2/spx_hp.csv')
df_final = pd.merge(df_rty, df_spx, left_index=True, right_index=True, how='inner')

In [9]:
df_final.head()

,px_last_rty,pct_change_rty,px_last_spx,pct_change_spx
Date,,,,
11/28/2023,1792.808,-0.463370,4554.89,0.098013
11/27/2023,1801.154,-0.351093,4550.43,-0.195423
11/24/2023,1807.500,0.666095,4559.34,0.059693
11/22/2023,1795.540,0.688457,4556.62,0.406109
11/21/2023,1783.263,-1.318201,4538.19,-0.202094


In [10]:
# function to calculate the correlations based on historical data
def corr_hp(df_final, horizon_list):

    corr1 = []
    df_final.index = pd.to_datetime(df_final.index)
    final_date = df_final.index[0]
    for i in horizon_list:
        date_hl =  final_date - pd.DateOffset(months=i)
        print(f"The {i} months date before {final_date} is : {date_hl}")

        df_hl = df_final[df_final.index >= date_hl]
        corr_hl = df_hl['pct_change_rty'].corr(df_hl['pct_change_spx'])
        print(f"The correlation for {i} months horizon is: {corr_hl:.6f} \n")
        corr1.append(corr_hl)

    return corr1

In [11]:
corr = corr_hp(df_final, [6, 12, 18, 24])

The 6 months date before 2023-11-28 00:00:00 is : 2023-05-28 00:00:00


The correlation for 6 months horizon is: 0.736798 

The 12 months date before 2023-11-28 00:00:00 is : 2022-11-28 00:00:00
The correlation for 12 months horizon is: 0.825055 

The 18 months date before 2023-11-28 00:00:00 is : 2022-05-28 00:00:00
The correlation for 18 months horizon is: 0.885226 

The 24 months date before 2023-11-28 00:00:00 is : 2021-11-28 00:00:00
The correlation for 24 months horizon is: 0.889623 



---

<div style="border-bottom: 2px solid black; display: inline-block; margin-bottom: 20px;">
<span style="font-family: Cambria, serif; font-weight: bold; font-size: 20px;">1) The Discount Factor Table after Linear Interpolation Looks like this:</span>
</div>

<div style="font-family: Cambria, serif;">

| Keys | Dates     | DF        | delta(T_x, T_0) | r(0,T_x)   | F(T_x1,T_x2)   |
|------|-----------|-----------|-----------------|------------| ------------|
| T_0  | 11/28/2023  | 1.000000  | 0.00000000     | 0.00000000 | -  |
| T_1  | 12/01/2023 | 0.999556  | 0.00821918     | 0.05403200 | -  |
| T_2  | 11/28/2025 | 0.911000  | 2.00000000     | <span style="background-color: green;">0.04660619</span> | -  |
| T_3  | 12/03/2025 | 0.910544  | 2.01369863     | 0.04653782 |<span style="background-color: green;">0.04650710</span> |

</div>

---

<div style="border-bottom: 2px solid black; display: inline-block;">
<span style="font-family: Cambria, serif; font-weight: bold; font-size: 20px;">2.1) The continuous dividend yield 'D1' for RTY Index from bloomberg data is 1.606% or 0.01606</span>
</div>

<div style="border-bottom: 2px solid black; display: inline-block;">
<span style="font-family: Cambria, serif; font-weight: bold; font-size: 20px;">2.2) The continuous dividend yield 'D2' for SPX Index from bloomberg data is 1.598% or 0.01598</span>
</div>

---

<div style="border-bottom: 2px solid black; display: inline-block; margin-bottom: 20px;">
<span style="font-family: Cambria, serif; font-weight: bold; font-size: 20px;">3) The Moneyness and Volatility Matrices Looks like this:</span>
</div>

<div style="font-family: Cambria, serif;">

| Moneyness | RTY Implied Volatility (24M) | SPX Implied Volatility (24M) |
|-----------|------------------| ------------------|
| 70.00     | 27.772%         | 25.440%         |
| 75.00     | 26.371%         | 24.080%         |
| 80.00     | 25.104%         | 22.833%         |
| 85.00     | 23.957%         | 21.676%         |
| 90.00     | 22.920%         | 20.542%         |
| 95.00     | 21.980%         | 19.392%         |
| 100.00    | 21.125%         | 18.195%         |

</div>

---

<div style="border-bottom: 2px solid black; display: inline-block; margin-bottom: 20px;">
<span style="font-family: Cambria, serif; font-weight: bold; font-size: 20px;">4) The correlation between indices (RTY and SPX) for 6M, 12M, 18M and 24M time horizons Looks like this:</span>
</div>

<div style="font-family: Cambria, serif;">

| Time horizon | Correlation Factor(rho12)   |
|------|-----------|
| 6M  | 0.736798  |
| 12M | 0.825055 |
| 18M  | 0.885226 |
| 24M | 0.889623 |

</div>

---

<div style="border-bottom: 2px solid black; display: inline-block;">
<span style="font-family: Cambria, serif; font-weight: bold; font-size: 28px;">V) Now Calculating the Value of the Note using Monte Carlo Simulations ==></span>
</div>

In [12]:
# note feature inputs that will go in the functions to calculate the value of the note

# Number of Trials to run Monte Carlo Simulation
Ntrials = 10
print(f"The number of Trial runs for Monte Carlo Simulations will be: {Ntrials}\n")

# Number of simulations
N = 100000
print(f"The number of Monte Carlo Simulations in each Trial will be: {N}\n")

# Initial Index Values
S10 = 1792.808
S20 = 4554.89
NA = 2
print(f"The number of underlying assets are: {NA}(RTY and SPX)")
print(f"The Initial Index Value for RTY Index is: {S10}\nThe Initial Index Value for SPX Index is: {S20}\n")

# Implied volatalities for each index at 70 Moneyness
sigma1 = 0.27772
sigma2 = 0.25440
print(f"The Implied Volatality for RTY Index at 70 Moneyness is: {sigma1}\nThe Implied Volatality for SPX Index at 70 Moneyness is: {sigma2}\n")

# 12 Month Correlation Factor between RTY and SPX indices
rho12 = 0.825055
print(f"The 12 Month Correlation Factor between RTY and SPX indices is: {rho12}\n")

# continously compounded risk-free rate(T_0 to T_2) for Stock-Tree simulation
r1 = r_T0_T2
print(f"The continously compounded risk-free rate for Stock-Tree simulation is: {r1:.8f}\n")

# continously compounded forward rate(T_1 to T_3) for discounting
r2 = F_T1_T3
print(f"The continously compounded risk-free rate which is used for discounting is: {r2:.8f}\n")

# continuous dividend yields for both indices
D1 = 0.01606
D2 = 0.01598
print(f"The continuous dividend yield for RTY Index is: {D1}\nThe continuous dividend yield for SPX Index is: {D2}\n")

# contingent coupon payment
cpn = 20
print(f"The contingent coupon payment is: ${cpn}\n")

# stated principal amount
principal = 1000
print(f"The stated principal amount is: ${principal}\n")

# coupon barriers for each index
CB1 = 0.70 * S10
CB2 = 0.70 * S20
print(f"The coupon barrier for RTY Index is: {CB1:.3f}\nThe coupon barrier for SPX Index is: {CB2:.3f}\n")

# downside threshold for each index
DT1 = 0.70 * S10
DT2 = 0.70 * S20
print(f"The downside threshold for RTY Index is: {DT1:.3f}\nThe downside threshold for SPX Index is: {DT2:.3f}\n")

The number of Trial runs for Monte Carlo Simulations will be: 10

The number of Monte Carlo Simulations in each Trial will be: 100000

The number of underlying assets are: 2(RTY and SPX)
The Initial Index Value for RTY Index is: 1792.808
The Initial Index Value for SPX Index is: 4554.89

The Implied Volatality for RTY Index at 70 Moneyness is: 0.27772
The Implied Volatality for SPX Index at 70 Moneyness is: 0.2544

The 12 Month Correlation Factor between RTY and SPX indices is: 0.825055

The continously compounded risk-free rate for Stock-Tree simulation is: 0.04660619

The continously compounded risk-free rate which is used for discounting is: 0.04650710

The continuous dividend yield for RTY Index is: 0.01606
The continuous dividend yield for SPX Index is: 0.01598

The contingent coupon payment is: $20

The stated principal amount is: $1000

The coupon barrier for RTY Index is: 1254.966
The coupon barrier for SPX Index is: 3188.423

The downside threshold for RTY Index is: 1254.966
T

In [13]:
# correlation matrix between RTY and SPX indices
rho = [[1, rho12],[rho12, 1]]
print(f"The correlation matrix between RTY and SPX indices is:\n{rho}\n")

# cholesky factorization for calculated correlation between RTY and SPX indices
C = np.linalg.cholesky(rho)
print(f"The cholesky factorization matrix between RTY and SPX indices is:\n{C}\n")

The correlation matrix between RTY and SPX indices is:
[[1, 0.825055], [0.825055, 1]]

The cholesky factorization matrix between RTY and SPX indices is:
[[1.         0.        ]
 [0.825055   0.56505243]]



In [14]:
# function to get fractional difference between two dates
def get_year_fraction(start_date, end_date):
    if isinstance(start_date, str):
        start_date = datetime.strptime(start_date, "%Y-%m-%d")
    if isinstance(end_date, str):
        end_date = datetime.strptime(end_date, "%Y-%m-%d")
    
    total_years = 0.0

    current_date = start_date
    while current_date < end_date:
        year = current_date.year
    
        if year == end_date.year:
            period_end = end_date
        else:
            period_end = datetime(year + 1, 1, 1)

        days_in_period = (period_end - current_date).days
        days_in_year = 366 if calendar.isleap(year) else 365
        total_years += days_in_period / days_in_year
        current_date = period_end
    
    return total_years

In [15]:
# The observation and coupon/redemption dates for the note
# observation dates list includes the starting date as well to account for the calculation of time period in between
observation_dates = [
    "2023-11-28",
    "2024-02-28",
    "2024-05-28",
    "2024-08-28",
    "2024-11-29",
    "2025-02-28",
    "2025-05-28",
    "2025-08-28",
    "2025-11-28"
]

cpn_rdm_dates = [
    "2024-03-04",
    "2024-05-31",
    "2024-09-03",
    "2024-12-04",
    "2025-03-05",
    "2025-06-02",
    "2025-09-03",
    "2025-12-03"
]

In [16]:
diff = get_year_fraction("2025-08-28", "2025-11-28")
diff

0.25205479452054796

In [17]:
# Observation Dates in fractional terms
M = []
for i in range(len(observation_dates) -1):
    diff = get_year_fraction(observation_dates[i], observation_dates[i+1])
    print(f"The difference between {observation_dates[i]} and {observation_dates[i+1]} is: {diff}")
    M.append(diff)

The difference between 2023-11-28 and 2024-02-28 is: 0.2516206302866981
The difference between 2024-02-28 and 2024-05-28 is: 0.2459016393442623
The difference between 2024-05-28 and 2024-08-28 is: 0.25136612021857924
The difference between 2024-08-28 and 2024-11-29 is: 0.2540983606557377
The difference between 2024-11-29 and 2025-02-28 is: 0.2490680440152706
The difference between 2025-02-28 and 2025-05-28 is: 0.24383561643835616
The difference between 2025-05-28 and 2025-08-28 is: 0.25205479452054796
The difference between 2025-08-28 and 2025-11-28 is: 0.25205479452054796


In [18]:
print(len(M), M)

8 [0.2516206302866981, 0.2459016393442623, 0.25136612021857924, 0.2540983606557377, 0.2490680440152706, 0.24383561643835616, 0.25205479452054796, 0.25205479452054796]


In [19]:
# Coupon diff dates to get the discounting date factor moving from redemption/coupon dates to observation dates
cpn_diff = []
for i in range(len(cpn_rdm_dates)):
    diff = get_year_fraction(observation_dates[i+1], cpn_rdm_dates[i])
    print(f"The difference between {observation_dates[i+1]} and {cpn_rdm_dates[i]} is: {diff}")
    cpn_diff.append(diff)

The difference between 2024-02-28 and 2024-03-04 is: 0.01366120218579235
The difference between 2024-05-28 and 2024-05-31 is: 0.00819672131147541
The difference between 2024-08-28 and 2024-09-03 is: 0.01639344262295082
The difference between 2024-11-29 and 2024-12-04 is: 0.01366120218579235
The difference between 2025-02-28 and 2025-03-05 is: 0.0136986301369863
The difference between 2025-05-28 and 2025-06-02 is: 0.0136986301369863
The difference between 2025-08-28 and 2025-09-03 is: 0.01643835616438356
The difference between 2025-11-28 and 2025-12-03 is: 0.0136986301369863


In [20]:
print(len(cpn_diff), cpn_diff)

8 [0.01366120218579235, 0.00819672131147541, 0.01639344262295082, 0.01366120218579235, 0.0136986301369863, 0.0136986301369863, 0.01643835616438356, 0.0136986301369863]


In [21]:
# sigma moneyness dict for both the indices
sigma1_moneyness = {
    70.00: 0.27772,
    100.00: 0.21125
}

sigma2_moneyness = {
    70.00: 0.25440,
    100.00: 0.18195
}

rho_map = {
    '6M': 0.736798,
    '12M': 0.825055,
    '18M': 0.885226,
    '24M': 0.889623,
}

In [22]:
# function definition to calculate the note value
def note_value(N, Ntrials, sigma1, sigma2, rho12):

    # initializing RTY and SPX index and Note payoff simulations
    S1 = np.zeros([N+1,len(M)+1])
    S2 = np.zeros([N+1,len(M)+1])
    V = np.zeros([N+1,len(M)+1])
    pathCV = np.zeros([N+1,len(M)+1])
    cpn_val = np.zeros([N+1,len(M)+1])
    hatCV_arr = np.zeros([N+1,len(M)+1])
    call_arr = np.zeros([N+1,len(M)+1])

    rho = [[1, rho12],[rho12, 1]]

    C = np.linalg.cholesky(rho)

    Note_vals = []
    for trial in range(1, Ntrials+1):
        # loop for stock simulationa and payoff at maturity
        for i in range(0, N+1):
            # initialization for both underlyings for each simulation n
            S1[i,0] = S10
            S2[i,0] = S20
            
            for j in range(1, len(M)+1):
                phi = []
                for z in range(0, NA):
                    phi.append(np.random.normal(0,1))
                diffusion = np.matmul(C,phi)
                S1[i,j] = S1[i,j-1]*np.exp(((r1-D1-0.5*sigma1**2)*M[j-1])+(sigma1*np.sqrt(M[j-1])*diffusion[0]))
                S2[i,j] = S2[i,j-1]*np.exp(((r1-D2-0.5*sigma2**2)*M[j-1])+(sigma2*np.sqrt(M[j-1])*diffusion[1]))

            # payoff at maturity
            # final value of either underlying index to be greater or equal than than 0.7 of initial level
            if (S1[i, len(M)] >= DT1) and (S2[i, len(M)] >= DT2):
                # discounting back from December 3rd 2025 to Final observation date November 28 2025
                V[i, len(M)] = (principal + cpn)*np.exp(-r2*cpn_diff[-1])

            # final value of either underlying index to be less than than 0.7 of initial level
            else:
                # to determine the worst performing index
                worst_performance_factor = np.minimum(((S1[i,len(M)])/(S1[i,0])), ((S2[i,len(M)])/(S2[i,0])))
                V[i, len(M)] = (principal * worst_performance_factor)*np.exp(-r2*cpn_diff[-1])

        #  loop for pathCV calculations and regressions
        for j in range(len(M)-1, 0, -1):
            for i in range(0, N+1):

                # getting the pathCV value by discounting back one period and appending that to pathCV list
                pathCV_val = np.exp(-r2*M[j])*V[i,j+1]
                pathCV[i,j] =  pathCV_val

                # add contingent coupon if both the underlying are above 0.7 of initial level
                if (S1[i, j] >= CB1) and (S2[i,j] >= CB2):
                    # discounting back the coupon from redemption date to observation date
                    cpn_sim = cpn*np.exp(-r2*cpn_diff[j-1])
                else:
                    cpn_sim = 0

                cpn_val[i,j] = cpn_sim

            x = S1[:, j]
            y = S2[:, j]
            z = pathCV[:, j]

            degree = 6
            # polynomial definition and running an OLS regressionn with S1 and S2 cubic polynomila as features and pathCV as target variable
            poly = PolynomialFeatures(degree=degree, include_bias=False)
            X = poly.fit_transform(np.column_stack([x, y]))

            X = sm.add_constant(X)

            model = sm.OLS(z, X)
            results = model.fit()

            # coefficients from model
            params = results.params

            # getting the value of hatCV by multiplying S1 and S2 cubic polynomial matrix with best fit coefficient matrix 
            hatCV = X @ params
            hatCV_arr[:, j] = hatCV

            # defining call amount and discounting that back from redemption date to observation date 
            call_amount = principal*np.exp(-r2*cpn_diff[j-1])
            call_arr[:, j] = call_amount

            for i in range(0, N+1):
                # considering call feature
                if (j>=2) and (hatCV_arr[i,j] > call_arr[i,j]):
                    V[i,j] = call_arr[i,j]
                else:
                    V[i,j] = pathCV[i,j]

                # adding the coupon
                V[i,j] += cpn_val[i,j]
            
                #print(i,j,S1[i,j],S2[i,j],V[i,j],call_arr[i,j],hatCV_arr[i,j])
        # loop for note value calculation
        t = 0
        for i in range(0, N+1): 
            t+=1
            # getting values at t=0
            V[i,0] = np.exp(-r2*M[0])*V[i,1]

        Note_Value = (np.sum(V[:,0])/t)
        Note_vals.append(Note_Value)
    return Note_vals

##### Calculating the note value using Ntrails=10, N = 100000 simulations at moneyness of (70,70) and (100,100) for (RTY,SPX) indices using 24N rho12=0.889623 and averaging the values to get final note value.

In [23]:
Ntrails=10
N=100000
sig1_70 = 0.27772
sig2_70 = 0.25440
sig1_100 = 0.21125
sig2_100 = 0.18195
rho12 = 0.889623

# calculating the note value list for Ntrials at moneyness of (70,70)
Note_val_70_70 = note_value(N, Ntrials, sig1_70, sig2_70, rho12)
print(f"The {Ntrials} Ntrials results for moneyness of (70,70) is:\n{Note_val_70_70}\n")

# calculating the note value list for Ntrials at moneyness of (100,100)
Note_val_100_100 = note_value(N, Ntrials, sig1_100, sig2_100, rho12)
print(f"The {Ntrials} Ntrials results for moneyness of (100,100) is:\n{Note_val_100_100}\n")

# averaging values for (70,70) and (100,100) at each trial
final_note_value = [(a + b)/2 for a, b in zip(Note_val_70_70, Note_val_100_100)]

# calculating the average between (70,70) and (100,100)
final_note_value_avg = np.mean(final_note_value)
print(f"The estimated fair value of the note by mean at moneyness (70,70) and (100,100) is: $ {final_note_value_avg:.3f}")

The 10 Ntrials results for moneyness of (70,70) is:
[948.9319468772472, 948.4833936106303, 947.377699471493, 946.5392640048569, 947.6002369464856, 947.7833533876744, 948.1555122314102, 949.4737676948556, 948.9884789756888, 948.4392176027192]

The 10 Ntrials results for moneyness of (100,100) is:
[990.8397894982442, 991.3245661190687, 990.7612521817648, 991.4235743252191, 991.4730531485346, 991.9394901115024, 991.45665189612, 992.1643721300372, 991.7324706492662, 991.8270188432932]

The estimated fair value of the note by mean at moneyness (70,70) and (100,100) is: $ 969.836


<div style="border-bottom: 2px solid black; display: inline-block; margin-bottom: 20px;">
<span style="font-family: Cambria, serif; font-weight: bold; font-size: 20px; background-color: green;">The estimated fair value of the note by mean at moneyness (70,70) and (100,100) is: $ 969.882</span>

</div>

#### Error Analysis taking the exact value of the note as $972.40

In [25]:
# creatinga dataframe to do error analysis
df = pd.DataFrame(final_note_value, columns=['Monte Values'])
df['Exact Note Value'] = 972.40
df['Errors'] = df['Monte Values'] - df['Exact Note Value']
df

,Monte Values,Exact Note Value,Errors
0,969.885868,972.4,-2.514132
1,969.903980,972.4,-2.496020
2,969.069476,972.4,-3.330524
3,968.981419,972.4,-3.418581
4,969.536645,972.4,-2.863355
5,969.861422,972.4,-2.538578
6,969.806082,972.4,-2.593918
7,970.819070,972.4,-1.580930
8,970.360475,972.4,-2.039525
9,970.133118,972.4,-2.266882


In [27]:
print(f"Mean Error is: {df['Errors'].mean()};\nand Standard Deviation of Errors is: {df['Errors'].std()}")

Mean Error is: -2.564244514694394;
and Standard Deviation of Errors is: 0.5523595314758891


### Here we see that the mean error is -2.564 and Standard deviation is 0.552. This is beacuse we do not know the volatalities of each underlying asset and the correlation factor. So for these fixed values we get the above results

---

<div style="border-bottom: 2px solid black; display: inline-block;">
<span style="font-family: Cambria, serif; font-weight: bold; font-size: 28px;">VI) Sensitivity Analysis ==></span>
</div>

<div style="border-bottom: 2px solid black; display: inline-block;">
<span style="font-family: Cambria, serif; font-weight: bold; font-size: 20px;">i) Varying simulations N from 1000 till 50000 for Ntrials=3, Moneyness is average between (70,70) and (100,100) and 24M rh012=0.889623 ==></span>
</div>

In [30]:
def note_N_sim(N):

    Ntrails=3
    sig1_70 = 0.27772
    sig2_70 = 0.25440
    sig1_100 = 0.21125
    sig2_100 = 0.18195
    rho12 = 0.889623

    # calculating the note value list for Ntrials at moneyness of (70,70)
    Note_val_70_70 = note_value(N, Ntrials, sig1_70, sig2_70, rho12)
    # averaging it out across trials
    avg_70_70 = np.mean(Note_val_70_70)

    # calculating the note value list for Ntrials at moneyness of (100,100)
    Note_val_100_100 = note_value(N, Ntrials, sig1_100, sig2_100, rho12)
    # averaging it out across trials
    avg_100_100 = np.mean(Note_val_100_100)

    # calculating the average between (70,70) and (100,100)
    final_note_value = (avg_70_70 + avg_100_100)/2
    output = {'Simulations': N, 'Monte': final_note_value}

    return output

In [ ]:
sim_sen = []
for i in range(1000, 21001, 5000):
    print(f"current N is: {i}")
    sim_sen.append(note_N_sim(i))

current N is: 1000
current N is: 6000
current N is: 11000
current N is: 16000
current N is: 21000
current N is: 26000


KeyboardInterrupt: 

In [104]:
sig1 = [0.27772, 0.21125]
sig2 = [0.25440, 0.18195]
rho_loop = [0.736798, 0.825055, 0.885226, 0.889623]

In [105]:
for i in rho_loop:
    for j in sig1:
        for k in sig2:
            note_value(j, k, i)

The estimated fair value of the note at moneyness of 70.0 for RTY and 70.0 for SPX with correlation of 6M using 100000 simulations and trial number 1 is: $931.847
The estimated fair value of the note at moneyness of 70.0 for RTY and 70.0 for SPX with correlation of 6M using 100000 simulations and trial number 2 is: $932.881
The estimated fair value of the note at moneyness of 70.0 for RTY and 70.0 for SPX with correlation of 6M using 100000 simulations and trial number 3 is: $932.243
The estimated fair value of the note at moneyness of 70.0 for RTY and 70.0 for SPX with correlation of 6M using 100000 simulations and trial number 4 is: $931.944
The estimated fair value of the note at moneyness of 70.0 for RTY and 70.0 for SPX with correlation of 6M using 100000 simulations and trial number 5 is: $932.436
The estimated fair value of the note at moneyness of 70.0 for RTY and 70.0 for SPX with correlation of 6M using 100000 simulations and trial number 6 is: $932.631
The estimated fair val

KeyboardInterrupt: 